# Drive mount

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Installs 

In [ ]:
!pip install -r requirements.txt

# Imports

In [ ]:
from art.attacks.evasion import ProjectedGradientDescent
from art.estimators.classification import KerasClassifier
import warnings
warnings.filterwarnings('ignore')

import json
import os

import cv2
from skimage import filters
from einops import rearrange
import statistics
import seaborn as sns; sns.set_theme()

import random
import traceback
import nibabel as nib
import scipy 

import numpy as np
from numpy import save
import matplotlib.pyplot as plt
import time
from datetime import datetime

import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import tensorflow_addons as tfa

from tensorflow.keras import layers
from plot_keras_history import plot_history

from sklearn.model_selection import ParameterGrid
from sklearn import metrics

from scripts.evalresults import *
from scripts.utils import *

# Adversarial Crafting

In [ ]:
!git clone https://github.com/Trusted-AI/adversarial-robustness-toolbox

In [ ]:
!pip install .

In [ ]:
# Load my train and validation sets with labels
x_train = np.load('./OASIS/healthy_oasis/x_train.npy')
y_train = np.load('./OASIS/healthy_oasis/y_train.npy')

x_train_adv = np.load('./OASIS/adv_oasis/x_train_adv.npy')

# All data shape: (H=256, W=256, C=1)

# Define the estimator (classifier) to generate the PGD attack 
model = tf.keras.applications.ResNet50(
  include_top=True,
  weights=None,
  input_tensor=None,
  input_shape=(256, 256,1),
  pooling=max,
  classes=3,
  classifier_activation='softmax'
)


# Compile
model.summary()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']);  

#model.load_weights('./saved/ResNet50/ckpt/Model_Ckpts_train.h5')



#------------------- Train the classifier--------------------------------------
# Callbacks          
calbks = tf.keras.callbacks.ModelCheckpoint(filepath='./saved/ResNet50/ckpt/Model_Ckpts_train.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)
 

# Train
hist = model.fit(x_train, y_train, epochs=50, validation_split=0.2, callbacks=[calbks]);
plot_history(hist, path='./saved/ResNet50/ckpt/History.png')
plt.close()


# Test
#loss_test, accuracy_test = model.evaluate(x_test, y_test)
#print('Accuracy on testing-set: {:4.2f}%'.format(accuracy_test * 100))
#-------------------------------------------------------------------------------




#------------------- Craft the adv samples -------------------------------------
# Create a ART keras classifier for the TensorFlow Keras model
classifier = KerasClassifier(model=model, clip_values=(0, 1))


# Get PGD attack on Guassian process classification
eps = 0.20
attack = ProjectedGradientDescent(classifier,
                                  eps=eps,
                                  eps_step=eps/10,
                                  targeted=True,
                                  batch_size=64)


# Generate adv examples
x_train_adv = attack.generate(x_train)
np.save('./saved/ResNet50/adv_data/x_train_adv.npy', x_train_adv)


# Evaluate on adversarial train data
loss_test, accuracy_test = model.evaluate(x_train_adv, y_train)
perturbation = np.mean(np.abs((x_train_adv - x_train)))
print(f'With eps=0.2 we have:\n')
print('Accuracy on adversarial train data: {:4.2f}%'.format(accuracy_test * 100))
print('Average perturbation: {:4.2f}\n'.format(perturbation))
#------------------------------------------------------------------------------


# Autoencoders

### Dense Convolutional Autoencoder

In [ ]:
#-- Model implementation : Dense Convolutional Autoencoder

def DCAE():
  
    input_img = tf.keras.Input(shape=(image_size, image_size, num_channels)) 

    x = layers.Conv2D(32 , 5, activation=layers.LeakyReLU(), strides=2, padding="same")(input_img)
    x = layers.Conv2D(64 , 5, activation=layers.LeakyReLU(), strides=2, padding="same")(x)
    x = layers.Conv2D(128, 5, activation=layers.LeakyReLU(), strides=2, padding="same")(x)    
    x = layers.Conv2D(128, 5, activation=layers.LeakyReLU(), strides=2, padding="same")(x)   
    x = layers.Conv2D(16 , 1, activation=layers.LeakyReLU(), strides=1, padding="same")(x)
    
    x = layers.Flatten()(x)
    encoded = layers.Dense(intermediate_dim, activation=layers.LeakyReLU())(x)

    #-- BOTTELNECK SIZE : 512 
    
    x = layers.Dense(16 * 16 * 16, activation=layers.LeakyReLU())(encoded)
    x = layers.Reshape((16, 16, 16))(x)

    x = layers.Conv2D(128, 1, strides=1, activation=layers.LeakyReLU(), padding="same")(x)    
    x = layers.Conv2DTranspose(128, 5, strides=2, activation=layers.LeakyReLU(), padding="same")(x) 
    x = layers.Conv2DTranspose(64 , 5, strides=2, activation=layers.LeakyReLU(), padding="same")(x)
    x = layers.Conv2DTranspose(32 , 5, strides=2, activation=layers.LeakyReLU(), padding="same")(x)
    x = layers.Conv2DTranspose(32 , 5, strides=2, activation=layers.LeakyReLU(), padding="same")(x)
    
    decoded = layers.Conv2D(num_channels, 1, activation=layers.LeakyReLU(), padding='same')(x)
    
    model = tf.keras.Model(input_img, decoded)

    return model

    
#-- Configure the hyperparameters

model_name = 'Dense Convolutional Autoencoder'
numEpochs = 50
learning_rate = 0.00001
image_size = 256
num_channels = 1
batch_size = 1
intermediate_dim = 512


#-- Configure training and testing Datasets 

saved_dir = './saved/'
data_dir  = './data/OASIS/'

test_healthy_path = './data/OASIS_all/OASIS_Flair_Test.npy'

test_path = './data/BraTS/s0/BraTS_Flair.npy'
brainmask_path = './data/BraTS/s0/BraTS_Brainmask.npy'
x_prior_path = './data/BraTS/s0/BraTS_prior_52.npy.npy'
label_path = './data/BraTS/s0/BraTS_GT.npy'

'''
#-- If using MSLUB as test-set
test_path = './data/MSLUB/MSLUB_Flair.npy'
brainmask_path = './data/MSLUB/MSLUB_Brainmask.npy'
x_prior_path = './data/MSLUB/MSLUB_prior_57.npy'
label_path = './data/MSLUB/MSLUB_GT.npy'
'''

list_len = [4805, 3078]    #-- BraTS and MSLUB test-set sizes, respectively.
len_testset = list_len[0]  #-- 0 for BraTS and 1 for MSLUB

train_paths = list_of_paths(data_dir)

nb_train_files = 66
data_gen = data_generator(train_paths[:nb_train_files], batch_size)
training_steps = (256 / batch_size) * nb_train_files

nb_val_files = 5
val_gen = data_generator(train_paths[-nb_val_files:], batch_size)
validation_steps = (256 / batch_size) * nb_val_files


#-- Checkpoints dir

date = datetime. now(). strftime("%Y_%m_%d-%I:%M:%S_%p")
ckpts_dir = os.path.join(saved_dir, f'Ckpts_{date}')
os.makedirs(ckpts_dir)
      
ckpts_path = os.path.join(ckpts_dir, 'Model_Ckpts.h5')
params_path = os.path.join(ckpts_dir, 'Parameters.txt')
results_path = os.path.join(ckpts_dir, 'Results.txt')
fig_path = os.path.join(ckpts_dir, 'History_plot.png')
dice_plot_path = os.path.join(ckpts_dir, 'Dice_plot.png')
predicted_path = os.path.join(ckpts_dir, 'Predicted.npy')
residual_path = os.path.join(ckpts_dir, 'Residuals.npy')
residual_BP_path = os.path.join(ckpts_dir, 'Residuals_BP.npy')
 

#-- Configure the training

opt = tf.keras.optimizers.Adam(learning_rate = learning_rate)
          
calbks = tf.keras.callbacks.ModelCheckpoint(filepath=ckpts_path, monitor='val_loss', save_best_only=True, save_weights_only=True, verbose=2)
tqdm_callback = tfa.callbacks.TQDMProgressBar() 

model = DCAE()
model.summary()
model.compile(optimizer=opt, loss='mae', metrics=['mse', SSIMLoss, MS_SSIMLoss])


#-- Print & Write model Parameters

parameters = (f'\nSelected model "{model_name}" with :\n - {batch_size}: Batche(s)\n - {numEpochs}: Epochs\n - {intermediate_dim}: Bottelneck size\n')
print(parameters)

           
#-- TRAIN 

print('\nTrain =>\n')
history = model.fit(x = data_gen,
                    steps_per_epoch = training_steps,
                    validation_data = val_gen,
                    validation_steps = validation_steps,
                    verbose = 0,
                    epochs = numEpochs,
                    callbacks = [calbks, tqdm_callback]
                    )

                          
#-- Get training and test loss histories 

plot_history(history, path=fig_path)
plt.close()
time.sleep(2)


#-- Test  

print('\nTest ===>\n')
my_test = np.load(test_path)
brainmask = np.load(brainmask_path)
x_prior = np.load(x_prior_path)
my_labels = np.load(label_path)

healthy_test = np.load(test_healthy_path)
steps = healthy_test.shape[0]

score = model.evaluate(x=healthy_test, y=healthy_test, verbose=0, steps=steps, callbacks = [tqdm_callback])    
score_out = (f'\nTest on healthy unseen data :\n - Test loss (MAE): {score[0]},\n - Test MSE : {score[1]},\n - Test SSIM : {score[2]}\n - Test MS_SSIM : {score[3]}\n')
print(score_out)


#-- Predict

print('\nPredict =====>\n')
predicted = model.predict(x=my_test, verbose=1, steps=len_testset)
np.save(predicted_path, predicted)
time.sleep(4)


#-- Calculate, Post-process and Save Residuals

print('\nCalculate, Post-process and Save Residuals =====>\n')     
residual_BP = calculate_residual_BP(my_test, predicted, brainmask)  #-- You can use either brainmask or x_prior
np.save(residual_BP_path, residual_BP)
        
residual = calculate_residual(my_test, predicted, brainmask)  #-- You can use either brainmask or x_prior
np.save(residual_path, residual)
        

#-- Evaluation

print('\nEvaluate =========>\n')        
[AUROC, AUPRC, AVG_DICE, MAD, STD], DICE = eval_residuals(my_labels, residual)     
results = (f'\nResults after median_filter :\n - AUROC = {AUROC}\n - AUPRC = {AUPRC}\n - AVG_DICE = {AVG_DICE}\n - MEDIAN_ABSOLUTE_DEVIATION = {MAD}\n - STANDARD_DEVIATION = {STD}')
print(results)
                      
plt.figure()
hor_axis = [x for x in range(len_testset)]
plt.scatter(hor_axis, DICE, s = 5, marker = '.', c = 'blue')
plt.ylabel('Dice Score')
plt.xlabel('N° Samples')
plt.title('Dice scores')
plt.savefig(dice_plot_path)
time.sleep(2)


#-- Save

print('\nSave Results and Parameters =============>\n')
f = open(results_path, "w")
f.write(results)       
f.close()   
                         
f = open(params_path, "w")
f.write(parameters)
f.write(score_out)
f.close()


#-- End

print('\nEnd !\n')

# Latent Variable Models

### Variational Autoencoder

In [ ]:
#-- Model implementation : Variational Autoencoder

class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon
        

def VAE():

  inputs = tf.keras.Input(shape=(image_size, image_size, num_channels))
  
  x = layers.Conv2D(32 , 5, activation=layers.LeakyReLU(), strides=2, padding="same")(inputs)
  x = layers.Conv2D(64 , 5, activation=layers.LeakyReLU(), strides=2, padding="same")(x)
  x = layers.Conv2D(128, 5, activation=layers.LeakyReLU(), strides=2, padding="same")(x)    
  x = layers.Conv2D(128, 5, activation=layers.LeakyReLU(), strides=2, padding="same")(x)   
  x = layers.Conv2D(16 , 1, activation=layers.LeakyReLU(), strides=1, padding="same")(x)   
  x = layers.Flatten()(x)
  encoded = layers.Dense(intermediate_dim, activation=layers.LeakyReLU())(x)   
  
  z_mean = layers.Dense(latent_dim, name="z_mean")(encoded)
  z_log_var = layers.Dense(latent_dim, name="z_log_var")(encoded)
  z = Sampling()([z_mean, z_log_var])
  
  x = layers.Dense(16 * 16 * 16, activation=layers.LeakyReLU())(z)
  x = layers.Reshape((16, 16, 16))(x)
  x = layers.Conv2D(128, 1, strides=1, activation=layers.LeakyReLU(), padding="same")(x)    
  x = layers.Conv2DTranspose(128, 5, strides=2, activation=layers.LeakyReLU(), padding="same")(x) 
  x = layers.Conv2DTranspose(64 , 5, strides=2, activation=layers.LeakyReLU(), padding="same")(x)
  x = layers.Conv2DTranspose(32 , 5, strides=2, activation=layers.LeakyReLU(), padding="same")(x)
  x = layers.Conv2DTranspose(32 , 5, strides=2, activation=layers.LeakyReLU(), padding="same")(x)
  decoder_outputs = layers.Conv2D(num_channels, 1, activation=layers.LeakyReLU(), padding='same')(x)
  
  model = tf.keras.Model(inputs, decoder_outputs)
  
  return model


#-- Configure the hyperparameters

model_name = 'Variational Autoencoder'
numEpochs = 50
learning_rate = 0.00001
rate = 0.  
image_size = 256
num_channels = 1
batch_size = 1
latent_dim = 512
intermediate_dim = 2048


#-- Configure training and testing Datasets 

saved_dir = './saved/'
data_dir  = './data/OASIS/'

test_healthy_path = './data/OASIS_all/OASIS_Flair_Test.npy'

test_path = './data/BraTS/s0/BraTS_Flair.npy'
brainmask_path = './data/BraTS/s0/BraTS_Brainmask.npy'
x_prior_path = './data/BraTS/s0/BraTS_prior_52.npy.npy' 
label_path = './data/BraTS/s0/BraTS_GT.npy'

'''
#-- If using MSLUB as test-set
test_path = './data/MSLUB/MSLUB_Flair.npy'
brainmask_path = './data/MSLUB/MSLUB_Brainmask.npy'
x_prior_path = './data/MSLUB/MSLUB_prior_57.npy'    
label_path = './data/MSLUB/MSLUB_GT.npy'
'''

list_len = [4805, 3078]    #-- BraTS and MSLUB test-set sizes, respectively.
len_testset = list_len[0]  #-- 0 for BraTS and 1 for MSLUB

train_paths = list_of_paths(data_dir)

nb_train_files = 66
data_gen = data_generator(train_paths[:nb_train_files], batch_size)
training_steps = (256 / batch_size) * nb_train_files

nb_val_files = 5
val_gen = data_generator(train_paths[-nb_val_files:], batch_size)
validation_steps = (256 / batch_size) * nb_val_files


#-- Checkpoints dir

date = datetime. now(). strftime("%Y_%m_%d-%I:%M:%S_%p")
ckpts_dir = os.path.join(saved_dir, f'Ckpts_{date}')
os.makedirs(ckpts_dir)
     
ckpts_path = os.path.join(ckpts_dir, 'Model_Ckpts.h5')
params_path = os.path.join(ckpts_dir, 'Parameters.txt')
results_path = os.path.join(ckpts_dir, 'Results.txt')
fig_path = os.path.join(ckpts_dir, 'History_plot.png')
dice_plot_path = os.path.join(ckpts_dir, 'Dice_plot.png')
predicted_path = os.path.join(ckpts_dir, 'Predicted.npy')
residual_path = os.path.join(ckpts_dir, 'Residuals.npy')
residual_BP_path = os.path.join(ckpts_dir, 'Residuals_BP.npy')

      
#-- Configure the training

opt = tf.keras.optimizers.Adam(learning_rate = learning_rate)
           
calbks = tf.keras.callbacks.ModelCheckpoint(filepath=ckpts_path, monitor='loss', save_best_only=True, save_weights_only=True, verbose=2)
tqdm_callback = tfa.callbacks.TQDMProgressBar() 

model = VAE()
model.summary()
model.compile(optimizer=opt, loss='mae', metrics=['mse', SSIMLoss, MS_SSIMLoss])


#-- Print & Write model Parameters

parameters = (f'\nSelected model "{model_name}" with :\n - {batch_size}: Batche(s)\n - {numEpochs}: Epochs\n - {intermediate_dim}: Intermediate dim\n - {latent_dim}: Bottelneck size\n')
print(parameters)

  
#-- TRAIN 

print('\nTrain =>\n')
history = model.fit(x = data_gen,
                    steps_per_epoch = training_steps,
                    validation_data = val_gen,
                    validation_steps = validation_steps,
                    verbose = 0,
                    epochs = numEpochs,
                    callbacks = [calbks, tqdm_callback]
                    )

                          
#-- Get training and test loss histories 

plot_history(history, path=fig_path)
plt.close()
time.sleep(2)


#-- Test

print('\nTest ===>\n')
my_test = np.load(test_path)
brainmask = np.load(brainmask_path)
x_prior = np.load(x_prior_path)
my_labels = np.load(label_path)

healthy_test = np.load(test_healthy_path)
steps = healthy_test.shape[0]

score = model.evaluate(x=healthy_test, y=healthy_test, verbose=0, steps=steps, callbacks = [tqdm_callback])    
score_out = (f'\nTest on healthy unseen data :\n - Test loss (MAE): {score[0]},\n - Test MSE : {score[1]},\n - Test SSIM : {score[2]}\n - Test MS_SSIM : {score[3]}\n')
print(score_out)


#-- Predict

print('\nPredict =====>\n')
predicted = model.predict(x=my_test, verbose=1, steps=len_testset)
np.save(predicted_path, predicted)
time.sleep(4)


#-- Calculate, Post-process and Save Residuals

print('\nCalculate, Post-process and Save Residuals =====>\n')     
residual_BP = calculate_residual_BP(my_test, predicted, brainmask)  #-- You can use either brainmask or x_prior
np.save(residual_BP_path, residual_BP)
        
residual = calculate_residual(my_test, predicted, brainmask)  #-- You can use either brainmask or x_prior
np.save(residual_path, residual)
        

#-- Evaluation

print('\nEvaluate =========>\n')        
[AUROC, AUPRC, AVG_DICE, MAD, STD], DICE = eval_residuals(my_labels, residual)     
results = (f'\nResults after median_filter :\n - AUROC = {AUROC}\n - AUPRC = {AUPRC}\n - AVG_DICE = {AVG_DICE}\n - MEDIAN_ABSOLUTE_DEVIATION = {MAD}\n - STANDARD_DEVIATION = {STD}')
print(results)
                      
plt.figure()
hor_axis = [x for x in range(len_testset)]
plt.scatter(hor_axis, DICE, s = 5, marker = '.', c = 'blue')
plt.ylabel('Dice Score')
plt.xlabel('N° Samples')
plt.title('Dice scores')
plt.savefig(dice_plot_path)
time.sleep(2)


#-- Save

print('\nSave Results and Parameters =============>\n')
f = open(results_path, "w")
f.write(results)       
f.close()   
                       
f = open(params_path, "w")
f.write(parameters)
f.write(score_out)
f.close()

      
#-- End

print('\nEnd !\n')

# Evaluate

### Evaluate on MSLUB Dataset

In [ ]:
list_ckpts_dir = ['./saved/DCAE/Predicted/mslub/', './saved/VAE/Predicted/mslub/'] 

for l in list_ckpts_dir:

  ckpts_dir = l
  results_path = os.path.join(ckpts_dir, 'Results.txt')
  dice_plot_path = os.path.join(ckpts_dir, 'Dice_plot.png')
  predicted_path = os.path.join(ckpts_dir, 'Predicted_mslub.npy')
  residual57_path = os.path.join(ckpts_dir, 'Residuals57.npy')
  residual84_path = os.path.join(ckpts_dir, 'Residuals84.npy')
  residual_BP_path = os.path.join(ckpts_dir, 'Residuals_BP.npy')

  test_path = './data/MSLUB/MSLUB_Flair.npy'
  prior57_path = './data/MSLUB/MSLUB_prior_57.npy'
  prior84_path = './data/MSLUB/MSLUB_prior_84.npy'
  label_path = './data/MSLUB/MSLUB_GT.npy'
  brainmask_path = './data/MSLUB/MSLUB_Brainmask.npy'

  # Test       
  print('\nTest ===>\n')
  predicted = np.load(predicted_path)
  my_test = np.load(test_path)
  brainmask = np.load(brainmask_path)
  my_labels = np.load(label_path)
  prior57 = np.load(prior57_path)
  prior84 = np.load(prior84_path)

  #-- Calculate, Post-process and Save Residuals
  print('\nCalculate, Post-process and Save Residuals =====>\n')     
  residual_BP = calculate_residual_BP(my_test, predicted, brainmask)
  np.save(residual_BP_path, residual_BP)
        
  residual_57 = calculate_residual(my_test, predicted, prior57)
  np.save(residual57_path, residual_57)

  residual_84 = calculate_residual(my_test, predicted, prior84)
  np.save(residual84_path, residual_84)
        

  #-- Evaluation
  print('\nEvaluate =========>\n')        
  [AUROC, AUPRC, AVG_DICE, MAD, STD], DICE = eval_residuals(my_labels, residual_57)     
  results_57 = (f'\nResults after median_filter and x_prior57 :\n - AUROC = {AUROC}\n - AUPRC = {AUPRC}\n - AVG_DICE = {AVG_DICE}\n - MEDIAN_ABSOLUTE_DEVIATION = {MAD}\n - STANDARD_DEVIATION = {STD}')
  print(results_57)
  
  [AUROC, AUPRC, AVG_DICE, MAD, STD], DICE = eval_residuals(my_labels, residual_84)     
  results_84 = (f'\nResults after median_filter and x_prior84 :\n - AUROC = {AUROC}\n - AUPRC = {AUPRC}\n - AVG_DICE = {AVG_DICE}\n - MEDIAN_ABSOLUTE_DEVIATION = {MAD}\n - STANDARD_DEVIATION = {STD}')
  print(results_84)
      
  len_testset = my_labels.shape[0]
               
  plt.figure()
  hor_axis = [x for x in range(len_testset)]
  plt.scatter(hor_axis, DICE, s = 5, marker = '.', c = 'blue')
  plt.ylabel('Dice Score')
  plt.xlabel('N° Samples')
  plt.title('Dice scores')
  plt.savefig(dice_plot_path)
  time.sleep(2)

  #-- Save
  print('\nSave Results and Parameters =============>\n')
  f = open(results_path, "w")
  f.write(results_57)
  f.write(results_84)       
  f.close()   
       
  print('\nEnd of evaluation !\n')

### Evaluate on BraTS Dataset

In [ ]:
list_ckpts_dir = ['./saved/DCAE/Predicted/brats/', './saved/VAE/Predicted/brats/']     


for l in list_ckpts_dir:

  ckpts_dir = l
  results_path = os.path.join(ckpts_dir, 'Results.txt')
  dice_plot_path = os.path.join(ckpts_dir, 'Dice_plot.png')
  predicted_path = os.path.join(ckpts_dir, 'Predicted_brats.npy')
  residual_path = os.path.join(ckpts_dir, 'Residuals.npy')
  residual_BP_path = os.path.join(ckpts_dir, 'Residuals_BP.npy')

  test_path = './data/BraTS/BraTS_Flair.npy'
  prior_path = './data/BraTS/BraTS_prior_52.npy'
  label_path = './data/BraTS/BraTS_GT.npy'
  brainmask_path = './data/BraTS/BraTS_Brainmask.npy'

  # Test       
  print('\nTest ===>\n')
  predicted = np.load(predicted_path)
  my_test = np.load(test_path)
  brainmask = np.load(brainmask_path)
  my_labels = np.load(label_path)
  prior = np.load(prior_path)

  #-- Calculate, Post-process and Save Residuals
  print('\nCalculate, Post-process and Save Residuals =====>\n')     
  residual_BP = calculate_residual_BP(my_test, predicted, brainmask)
  np.save(residual_BP_path, residual_BP)
        
  residual = calculate_residual(my_test, predicted, prior)
  np.save(residual_path, residual)
        

  #-- Evaluation
  print('\nEvaluate =========>\n')        
  [AUROC, AUPRC, AVG_DICE, MAD, STD], DICE = eval_residuals(my_labels, residual)     
  results = (f'\nResults after median_filter :\n - AUROC = {AUROC}\n - AUPRC = {AUPRC}\n - AVG_DICE = {AVG_DICE}\n - MEDIAN_ABSOLUTE_DEVIATION = {MAD}\n - STANDARD_DEVIATION = {STD}')
  print(results)
      
  len_testset = my_labels.shape[0]
               
  plt.figure()
  hor_axis = [x for x in range(len_testset)]
  plt.scatter(hor_axis, DICE, s = 5, marker = '.', c = 'blue')
  plt.ylabel('Dice Score')
  plt.xlabel('N° Samples')
  plt.title('Dice scores')
  plt.savefig(dice_plot_path)
  time.sleep(2)

  #-- Save
  print('\nSave Results and Parameters =============>\n')
  f = open(results_path, "w")
  f.write(results)       
  f.close()   
       
  #-- End
  print('\nEnd of evaluation !\n')